In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [3]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
3604,2022-01-31,33,MOZS,23,RSAN
3605,2022-01-31,38,RSAN,23,BOT
3606,2022-01-31,111,ZAMZ,23,ZAMC
3607,2022-01-31,111,ZIM,23,ZAMZ


In [4]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2022-02-02 19    258.94    63.3  2022      2    2            2  16390.902   
           20    258.94    63.9  2022      2    2            2  16546.266   
           21     75.18   113.0  2022      2    2            2   8495.340   
           22     50.03    97.6  2022      2    2            2   4882.928   
           23     46.09   136.0  2022      2    2            2   6268.240   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2022-02-02 19     evening  
           20     evening  
           21    standard  
           22         off  
           23         off  

[52392 rows x 8 columns]

In [5]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
375680,2022-01-31,23,33,MOZS,RSAN,2022,1,31,0,39.99,1319.67
375681,2022-01-31,23,38,RSAN,BOT,2022,1,31,0,39.99,1519.62
375682,2022-01-31,23,111,ZAMZ,ZAMC,2022,1,31,0,39.99,4438.89
375683,2022-01-31,23,111,ZIM,ZAMZ,2022,1,31,0,39.99,4438.89


In [6]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
375680,2022-01-31,23,-33,-1319.67,39.99,MOZS,2022,1,31,0
375681,2022-01-31,23,-38,-1519.62,39.99,RSAN,2022,1,31,0
375682,2022-01-31,23,-111,-4438.89,39.99,ZAMZ,2022,1,31,0
375683,2022-01-31,23,-111,-4438.89,39.99,ZIM,2022,1,31,0


In [7]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [8]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week   index  flow    value  \
date       hour                                                               
2016-01-01 0     ZAMC  2016      1    1            4       0   160  6416.00   
           0     ZAMZ  2016      1    1            4       1    10   401.00   
           0      ZIM  2016      1    1            4       1  -170 -6817.00   
           1     ZAMC  2016      1    1            4       2   160  6412.80   
           1     ZAMZ  2016      1    1            4       5    10   400.80   
...               ...   ...    ...  ...          ...     ...   ...      ...   
2022-01-31 22     MOZ  2022      1   31            0  751343  -112 -5533.92   
           22    ZAMC  2022      1   31            0  375674   117  5780.97   
           23     LES  2022      1   31            0  375678    -5  -199.95   
           23     MOZ  2022      1   31            0  751359  -106 -4238.94   
           23    ZAMC  2022      1   31            0  375682   111  4438.89   

                 price  cat  
date       hour              
2016-01-01 0     40.10  off  
           0     80.20  off  
           0     40.10  off  
           1     40.08  off  
           1     80.16  off  
...                ...  ...  
2022-01-31 22    98.82  off  
           22    49.41  off  
           23    39.99  off  
           23    79.98  off  
           23    39.99  off  

[215935 rows x 10 columns]

In [9]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2016 1     BOT  evening  supply        -25   -1501.40           2   
                morning  supply       -285  -14572.40           4   
                off      supply      -1190  -55919.14          41   
                standard supply      -1165  -57809.40          18   
           HCB  evening  supply        -57   -6204.59           2   
...                                    ...        ...         ...   
2022 1     ZIM  evening  supply         -3    -776.79           3   
                off      consume      8186  222385.97          91   
                         supply         -1     -45.17           1   
                standard consume      3726  252930.71          87   
                         supply         -5    -375.44           5   

                                  price_weight_mean   price_mean      power  
year month node cat      side                                                
2016 1     BOT  evening  supply           60.056000    60.050000 -12.500000  
                morning  supply           51.131228    55.290000 -71.250000  
                off      supply           46.990874    48.856341 -29.024390  
                standard supply           49.621803    56.768333 -64.722222  
           HCB  evening  supply          108.852456    99.140000 -28.500000  
...                                             ...          ...        ...  
2022 1     ZIM  evening  supply          258.930000  1294.650000  -1.000000  
                off      consume          27.166622   137.064945  89.956044  
                         supply           45.170000   225.850000  -1.000000  
                standard consume          67.882638   361.062644  42.827586  
                         supply           75.088000   375.440000  -1.000000  

[2664 rows x 6 columns]

In [10]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')